In [4]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random

In [38]:
random.seed(1356) # set seed so that each play is the same

14024079292171647447

In [5]:
def hex_add(a, b):
    return Hex[a[0] + b[0], a[1] + b[1], a[2] + b[2]]

def hex_subtract(a, b):
    return Hex[a[0] - b[0], a[1] - b[1], a[2] - b[2]]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_directions[direction])

def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex[0]) + abs(hex[1]) + abs(hex[2])) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

In [6]:
hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

In [142]:
random.seed(1356)
side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 3 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4
side

[14024079292171647447, 15846705576740111]

In [176]:
def initializeBoard():
    Hex = {}

    for x in range(-N, N):
        for y in range(max(-N, -x-N) , min(+N, -x+N)):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

board = initializeBoard()
board

{(-3, 0, 3): '',
 (-3, 1, 2): '',
 (-3, 2, 1): '',
 (-2, -1, 3): '',
 (-2, 0, 2): '',
 (-2, 1, 1): '',
 (-2, 2, 0): '',
 (-1, -2, 3): '',
 (-1, -1, 2): '',
 (-1, 0, 1): '',
 (-1, 1, 0): '',
 (-1, 2, -1): '',
 (0, -3, 3): '',
 (0, -2, 2): '',
 (0, -1, 1): '',
 (0, 0, 0): '',
 (0, 1, -1): '',
 (0, 2, -2): '',
 (1, -3, 2): '',
 (1, -2, 1): '',
 (1, -1, 0): '',
 (1, 0, -1): '',
 (1, 1, -2): '',
 (2, -3, 1): '',
 (2, -2, 0): '',
 (2, -1, -1): '',
 (2, 0, -2): ''}

In [87]:
def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N):
        for y in range(max(-N, -x-N) , min(+N, -x+N)):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

# wzobristboard

In [86]:
def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N):
        for y in range(max(-N, -x-N) , min(+N, -x+N)):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard


# bzobristboard

In [85]:
def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N):
        for y in range(max(-N, -x-N) , min(+N, -x+N)):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex


# zboard

In [140]:
def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    
    for cel in board:
        if board[cel] == 14024079292171647447:
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        else:
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    
    return zobristkey

boardToZobrist(board, zboard, wzobristboard, bzobristboard)

7587650002905401167

In [129]:
board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

In [78]:
def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

In [79]:
# Form groups
# score = find_scores(board, 'w')
def findGroups(score, board):
    new_list = []
    group = {}
    group_state = {}
    group_count = 0

    for tile in score:
        counter = 0
        group[group_count] = 0
        if not tile in new_list:
            for direction in hex_directions:
                new_tile = tuple(map(sum, zip(tile, direction)))
                if new_tile in score:
                    new_list += [new_tile] # Already seen in group

                if score[tile] == board[direction]:
                    group[group_count] += counter
                    group_state.setdefault(tile, []).append(new_tile)

                counter += 1
        group_count += 1
    return group_state

In [80]:
# Check if lists can be grouped by at least 1 common member.
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 

In [81]:
# Count total of each group
# group = findGroups(score, board)

def totalgroups(group):
    count_list = []

    for tile in group:
        for tile2 in group:
            if tile != tile2:
                intersect = list(Intersection(group[tile], group[tile2]))

                if len(intersect) != 0:
                    count_list += [len(list(set(group[tile] + group[tile2])))]
    return count_list


# print(totalgroups(group))

In [89]:
score = find_scores(board, 14024079292171647447) #score of player1
score2 = find_scores(board, 15846705576740111) #score of player2

group = findGroups(score, board)
group2 = findGroups(score2, board)
print(totalgroups(group), totalgroups(group2))

[3, 3] [7, 7, 7, 7]


In [25]:
# multiply all group counts of the same color for final game score
from functools import reduce

count_list = totalgroups(group)
count_list2 = totalgroups(group2)

def multiplyall(count_list):
    return reduce(lambda x, y: x*y, count_list)

game_total = multiplyall(count_list)
# game_total2 = multiplyall(count_list2)
game_total#, game_total2

9

In [110]:
# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [111]:
# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [120]:
def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

In [32]:
~(len(board[(0,0,0)]) < 0)

-1

In [33]:
#Check if a move is valid
def validMove(x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [180]:
#Creating entire the game tree for the play
from copy import deepcopy
import time
def buildGameTree0(board, depth, samples):
    depth += 1
    gametree = {}
    tempscore = 10
    
#     board0 = board # board with assigned black and white pieces
    zboard = initializeBoardZobrist() # initial board, without any pieces

    wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
    bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent
    
    visited = [] # Make sure you haven't done this move yet
    for sample in range(samples):
        board2 = deepcopy(board) # make sure that you copy the board from the parent!!

        #Only make a move when a move is valid
        valid = False
        while not valid:
            x, y = random.randint(-N,N), random.randint(-N,N)
            if not (x, y, -x-y) in visited:
                valid = validMove(x, y)
        makeMove(board2, [x, y], 'b')
        visited += [(x, y, -x-y)]

        #Only make a move when a move is valid 2
        valid = False
        while not valid:
            x2, y2 = random.randint(-N,N), random.randint(-N,N)
            if not (x2, y2, -x2-y2) in visited:
                valid = validMove(x2, y2)
        makeMove(board2, [x2, y2], 'w')
        visited += [(x2, y2, -x2-y2)]

#         board2 = initializeBoard() # board with assigned black and white pieces
        zboard = initializeBoardZobrist() # initial board, without any pieces

        wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
        bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

        zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)
        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore]) # store full board + temporary score + horizon effect?
                     
    return gametree

start = time.time()
g = buildGameTree0(board, 0, 3)#13)
end = time.time()
print('Time: ', end - start)
g

Time:  0.0016968250274658203


{2819087354705866244: [[10726909059677089085,
   {(-3, 0, 3): '',
    (-3, 1, 2): '',
    (-3, 2, 1): '',
    (-2, -1, 3): '',
    (-2, 0, 2): '',
    (-2, 1, 1): '',
    (-2, 2, 0): '',
    (-1, -2, 3): '',
    (-1, -1, 2): '',
    (-1, 0, 1): '',
    (-1, 1, 0): '',
    (-1, 2, -1): 'w',
    (0, -3, 3): '',
    (0, -2, 2): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (0, 2, -2): '',
    (1, -3, 2): '',
    (1, -2, 1): '',
    (1, -1, 0): '',
    (1, 0, -1): '',
    (1, 1, -2): '',
    (2, -3, 1): 'b',
    (2, -2, 0): '',
    (2, -1, -1): '',
    (2, 0, -2): ''},
   1,
   10],
  [17164897749536883520,
   {(-3, 0, 3): '',
    (-3, 1, 2): '',
    (-3, 2, 1): '',
    (-2, -1, 3): 'w',
    (-2, 0, 2): '',
    (-2, 1, 1): '',
    (-2, 2, 0): '',
    (-1, -2, 3): '',
    (-1, -1, 2): '',
    (-1, 0, 1): '',
    (-1, 1, 0): '',
    (-1, 2, -1): '',
    (0, -3, 3): '',
    (0, -2, 2): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (0, 2, -2): '',
    

In [178]:
# Build the next layers
totalTree = {}
for board_zobrist in g:
    print(board_zobrist, g[board_zobrist])# [identity, tree, depth, score]
    break
#     newdepth = buildGameTree(g[board_zobrist][1], depth, samples)
#     totalTree.setdefault(depth+1, []).append(newdepth)

5474507440992383054 [[729939849913862877, {(-3, 0, 3): '', (-3, 1, 2): '', (-3, 2, 1): '', (-2, -1, 3): '', (-2, 0, 2): '', (-2, 1, 1): '', (-2, 2, 0): '', (-1, -2, 3): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (-1, 2, -1): '', (0, -3, 3): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (0, 2, -2): '', (1, -3, 2): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): '', (1, 1, -2): '', (2, -3, 1): '', (2, -2, 0): '', (2, -1, -1): '', (2, 0, -2): ''}, 1, 10], [15872109129810816640, {(-3, 0, 3): '', (-3, 1, 2): '', (-3, 2, 1): '', (-2, -1, 3): '', (-2, 0, 2): '', (-2, 1, 1): '', (-2, 2, 0): '', (-1, -2, 3): '', (-1, -1, 2): '', (-1, 0, 1): '', (-1, 1, 0): '', (-1, 2, -1): '', (0, -3, 3): '', (0, -2, 2): '', (0, -1, 1): '', (0, 0, 0): '', (0, 1, -1): '', (0, 2, -2): '', (1, -3, 2): '', (1, -2, 1): '', (1, -1, 0): '', (1, 0, -1): '', (1, 1, -2): '', (2, -3, 1): '', (2, -2, 0): '', (2, -1, -1): '', (2, 0, -2): ''}, 1, 10], [16846244176120008955, {(-3, 0, 3): '', (-3

In [94]:
gametree['hi']['do'] = [{'z': 'l'} ,9,8]
gametree

{'hi': {'do': [{'z': 'l'}, 9, 8]}}

In [ ]:
def getComputerMove(board, computerLetter):
    # Given a board and the computer's letter, determine where to move and return that move.
    if computerLetter == 'b':
        playerLetter = 'w'
    else:
        playerLetter = 'b'

# Checking Whether the Computer Can Win in One Move

In [ ]:
# Here is the algorithm for our Tic-Tac-Toe AI:
# First, check if we can win in the next move.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, computerLetter, i)
    if isWinner(boardCopy, computerLetter):
        return i

# Checking Whether the Player Can Win in One Move

In [ ]:
# Check if the player could win on their next move and block them.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, playerLetter, i)
    if isWinner(boardCopy, playerLetter):
        return i

In [ ]:
#Checking the Corner, Center, and Side Spaces (in That Order)

In [ ]:
# Try to take one of the corners, if they are free.
move = chooseRandomMoveFromList(board, [1, 3, 7, 9])
if move != None:
    return move

In [ ]:
# If all the corner spaces are taken, the chooseRandomMoveFromList() function returns None, and the execution moves on to line 113:
# Try to take the center, if it is free.
if isSpaceFree(board, 5):
    return 5

In [ ]:
#If none of the corners is available, line 114 moves on the center space if it is free. If the center space isn’t free, the execution moves on to line 117:
# Move on one of the sides.
return chooseRandomMoveFromList(board, [2, 4, 6, 8])

# Checking Whether the Board Is Full

In [ ]:
def isBoardFull(board):
# Return True if every space on the board has been taken. Otherwise, return False.
for i in range(1, 10):
    if isSpaceFree(board, i):
        return False
    return True

# The Game Loop

In [ ]:
print('Welcome to Tic-Tac-Toe!')

In [ ]:
Hex

In [ ]:
# Choosing the Player’s Mark and Who Goes First
playerLetter, computerLetter = inputPlayerLetter()
turn = whoGoesFirst()
print('The ' + turn + ' will go first.')
gameIsPlaying = True

In [ ]:
while gameIsPlaying:
    if turn == 'player':
        # Player's turn
        drawBoard(theBoard)
        move = getPlayerMove(theBoard)
        makeMove(theBoard, playerLetter, move)

    if isWinner(theBoard, playerLetter):
        drawBoard(theBoard)
        print('Hooray! You have won the game!')
        gameIsPlaying = False

    else:
        if isBoardFull(theBoard):
            drawBoard(theBoard)
            print('The game is a tie!')
            break

        else:
            turn = 'computer'

    else:
        # Computer's turn
        move = getComputerMove(theBoard, computerLetter)
        makeMove(theBoard, computerLetter, move)
        if isWinner(theBoard, computerLetter):
            drawBoard(theBoard)
            print('The computer has beaten you! You lose.')
            gameIsPlaying = False
        else:
            if isBoardFull(theBoard):
                drawBoard(theBoard)
                print('The game is a tie!')
                break
            else:
                turn = 'player'

# Asking the Player to Play Again

In [ ]:
print('Do you want to play again? (yes or no)')
if not input().lower().startswith('y'):
    break

In [ ]:
# def printBoard(board):
#     line = ''
#     space = 0
#     for y in range(0, N):
#         for x in range(-N, N*2):
#             try:
#                 line += Hex[(x, y, -x-y)]
#             except:
#                 line += '_'
        
#     return line
# printBoard(Hex)